In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('../input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
for columns in df.columns:
    print(columns,"=",df[columns].nunique())

In [ ]:
a=df.drop(columns=['StandardHours','EmployeeCount','Over18','EmployeeNumber'])

In [ ]:
a.head()


In [ ]:
object_col = []
for column in a.columns:
    if a[column].dtype == object and len(a[column].unique()) <= 30:
        object_col.append(column)
        print(f"{column} : {a[column].unique()}")
        print(a[column].value_counts())
        print("====================================")
object_col.remove('JobRole' )
object_col.remove('EducationField' )  
print(object_col)
        
        

In [ ]:
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
a["Attrition"] = label.fit_transform(a.Attrition)

In [ ]:
cont_col = []
for column in a.columns:
    if a[column].dtypes != object and a[column].nunique() > 30:
        print(f"{column} : Minimum: {a[column].min()}, Maximum: {a[column].max()}")
        cont_col.append(column)
        print("====================================")


In [ ]:
plt.figure(figsize=(20, 10))
for i, column in enumerate(cont_col, 1):

    plt.subplot(2, 4, i)
    
    sns.kdeplot(data = a[a['Attrition'] == 0], x = column, shade = True,  alpha = 1, color = '#512b58',Label='Attrition = NO' )
    sns.kdeplot(data = a[a['Attrition'] == 1], x = column, shade = True,  alpha = 0.8, color = '#fe346e',Label='Attrition = YES')
    plt.legend()
    

In [ ]:
disc_col = []
for column in a.columns:
    if a[column].dtypes != object and a[column].nunique() < 30:
        print(f"{column} : {a[column].unique()}")
        disc_col.append(column)
        print("====================================")
disc_col.remove('Attrition')

In [ ]:
plt.figure(figsize=(20, 20))

for i, column in enumerate(disc_col, 1):
    plt.subplot(4, 4, i)
    a[a["Attrition"] == 0][column].hist(bins=35, color='blue', label='Attrition = NO', alpha=0.6)
    a[a["Attrition"] == 1][column].hist(bins=35, color='red', label='Attrition = YES', alpha=0.6)
    plt.legend()
    plt.grid(False)
    plt.xlabel(column)

In [ ]:

plt.figure(figsize=(20, 25))
for i, column in enumerate(object_col , 1):
    plt.subplot(5, 4, i)
  
    a[a["Attrition"] == 0][column].hist(bins=5,color='orange', label='Attrition = NO', alpha=0.6)
    a[a["Attrition"] == 1][column].hist(bins=5, color='red', label='Attrition = YES', alpha=0.6)
    plt.legend()
    plt.grid(False)
    plt.xlabel(column)

In [ ]:
    plt.figure(figsize=(20,5 ))
    a[a["Attrition"] == 0]["JobRole"].hist(bins=33,color='blue', label='Attrition = NO', alpha=0.6)
    a[a["Attrition"] == 1]["JobRole"].hist(bins=33, color='darkred', label='Attrition = YES', alpha=0.6)
    plt.legend()
    plt.grid(False)
    plt.xlabel("JobRole")

In [ ]:
    plt.figure(figsize=(10,5 ))
    a[a["Attrition"] == 0]["EducationField"].hist(bins=33,color='blue', label='Attrition = NO', alpha=0.6)
    a[a["Attrition"] == 1]["EducationField"].hist(bins=33, color='darkred', label='Attrition = YES', alpha=0.6)
    plt.legend()
    plt.grid(False)
    plt.xlabel("EducationField")

In [ ]:
a

In [ ]:
col = a.corr().nlargest(20, "Attrition").Attrition.index
plt.figure(figsize=(15, 15))
sns.heatmap(df[col].corr(), annot=True, cmap="RdYlGn", annot_kws={"size":10})

In [ ]:
dummy_col = [column for column in a.drop('Attrition', axis=1).columns if a[column].nunique() < 20]
data = pd.get_dummies(a, columns=dummy_col, drop_first=True, dtype='uint8')
data.info()

In [ ]:
print(data.columns)

In [ ]:
x = data.drop('Attrition', axis=1)
y = data.Attrition
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state= 0)

In [ ]:
from sklearn.preprocessing import StandardScaler 
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, classification_report, roc_curve, plot_roc_curve, auc, precision_recall_curve, plot_precision_recall_curve, average_precision_score
from sklearn.model_selection import cross_val_score

In [ ]:
models = []
models.append(['Logistic Regreesion', LogisticRegression(random_state=0)])
models.append([' DecisionTreeClassifier',  DecisionTreeClassifier(random_state=0)])
models.append(['Random Forest', RandomForestClassifier(random_state=0)])
lst_1= []

for m in range(len(models)):
    lst_2= []
    model = models[m][1]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    cm = confusion_matrix(y_test, y_pred)  #Confusion Matrix
    accuracies = cross_val_score(estimator = model, X = x_train, y = y_train, cv = 10)   #K-Fold Validation
    roc = roc_auc_score(y_test, y_pred)  #ROC AUC Score
    precision = precision_score(y_test, y_pred)  #Precision Score
    recall = recall_score(y_test, y_pred)  #Recall Score
    f1 = f1_score(y_test, y_pred)  #F1 Score
   
    
    print(models[m][0],':')
    print(cm)
    print('Accuracy Score: ',accuracy_score(y_test, y_pred))
    print('')
    print("K-Fold Validation Mean Accuracy: {:.2f} %".format(accuracies.mean()*100))
    print('')
    print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))
    print('')
    print('ROC AUC Score: {:.2f}'.format(roc))
    print('')
    print('Precision: {:.2f}'.format(precision))
    print('')
    print('Recall: {:.2f}'.format(recall))
    print('')
    print('F1: {:.2f}'.format(f1))
    print('-----------------------------------')
    print('')
    lst_2.append(models[m][0])
    lst_2.append((accuracy_score(y_test, y_pred))*100) 
    lst_2.append(accuracies.mean()*100)
    lst_2.append(accuracies.std()*100)
    lst_2.append(roc)
    lst_2.append(precision)
    lst_2.append(recall)
    lst_2.append(f1)
    lst_1.append(lst_2)



In [ ]:
from sklearn.model_selection import GridSearchCV
grid_models = [(LogisticRegression(),[{'C':[0.25,0.5,0.75,1],'random_state':[0]}]),(RandomForestClassifier(),[{'n_estimators':[100,150,200],'criterion':['gini','entropy'],'random_state':[0]}])]

In [ ]:
for i,j in grid_models:
    grid = GridSearchCV(estimator=i,param_grid = j, scoring = 'accuracy',cv = 10)
    grid.fit(x_train, y_train)
    best_accuracy = grid.best_score_
    best_param = grid.best_params_
    print('{}:\nBest Accuracy : {:.2f}%'.format(i,best_accuracy*100))
    print('Best Parameters : ',best_param)
    print('')
    print('----------------')
    print('')